In [1]:
import numpy as np
import scipy as sp
import scipy.stats as sps
import gc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from dateutil.parser import parse
from datetime import datetime, timedelta
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/data/')

План:

1) CRM: с qualified надо подтянуть
* имя 
* фамилия
* должность
* сайт
* телефон
* биллинг
* account_name
* Время первого попадания
* Время qualified
* разница между этими датами в днях
* Количество дней, недель, месяцев со статуса qualified

2) Из бекофиса подтянуть email и телефон, puid, inn

3) по puid из spark подтянуть информацию о компании (Industry, name, phones и т д) из спарка на месте

4) Потребление по аккаунту и сабаккаунтам по кварталам

* Количество аккаунтов в 1 месяц, 2 месяц, 3 месяц 4 месяц, и т д и 6+
* Потребление аккаунтом в течение последних 4 кварталов
* Суммарное потребление по сабаакунтам также
* количество сабаккунтов по кварталам
* среднее потребление сабаккаунтом в квартал

In [2]:
leads_table = lib.find_tables_in_hahn_folder(
    "//home/cloud_analytics/dwh/raw/crm/leads")[-1]
billing_leads_keys = lib.find_tables_in_hahn_folder(
    "//home/cloud_analytics/dwh/raw/crm/leads_billing_accounts")[-1]
billing_table = lib.find_tables_in_hahn_folder(
    "//home/cloud_analytics/dwh/raw/crm/billingaccounts")[-1]
email_leads_keys = lib.find_tables_in_hahn_folder(
    "//home/cloud_analytics/dwh/raw/crm/email_addr_bean_rel")[-1]
email_table = lib.find_tables_in_hahn_folder(
    "//home/cloud_analytics/dwh/raw/crm/email_addresses")[-1]

In [3]:
core_req = f"""
SELECT
    billing_account_id,
    ba_time as billing_created_time,
    mail_billing,
    mail_tech,
    mail_testing,
    mail_feature,
    mail_promo,
    mail_info,
    mail_event
FROM (
    SELECT
        billing_account_id,
        ba_time,
        mail_billing,
        mail_tech,
        mail_testing,
        mail_feature,
        mail_promo,
        mail_info,
        mail_event
    FROM (
        SELECT 
            billing_account_id,
            toDate(event_time) as ba_time,
            mail_billing,
            mail_tech,
            mail_testing,
            mail_feature,
            mail_promo,
            mail_info,
            mail_event
        FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
        WHERE isNotNull(billing_account_id)
        AND is_var == 'var'
        AND event == 'ba_created'
        AND billing_account_id != ''
        AND puid != ''
        AND master_account_id == ''
    ) as cube_information
    LEFT JOIN (
        SELECT
            billing_account_id,
            max(date_qualified) as date_qualified
        FROM (
            SELECT
                id,
                toDate(date_modified) as date_qualified
            FROM "{leads_table}"
            WHERE lead_source == 'var'
            and status = 'Converted'
        ) as a
        INNER JOIN (
            SELECT
                DISTINCT
                leads_id,
                billing_account_id
            FROM "{billing_leads_keys}" as a
            INNER JOIN (
                SELECT
                    id,
                    ba_id as billing_account_id
                FROM "{billing_table}"
            ) as b
            ON a.billingaccounts_id == b.id
        ) as b
        ON a.id == b.leads_id
        GROUP BY billing_account_id
    ) as  new
    ON cube_information.billing_account_id == new.billing_account_id
)
FORMAT TabSeparatedWithNames
"""
core_df = lib.execute_query(core_req)

## 0 уже дано

In [4]:
prev_req = """
SELECT
    *
FROM "//home/cloud_analytics/lunin-dv/additional_tickets_information/CLOUDANA-781"
FORMAT TabSeparatedWithNames
"""
prev_df = lib.execute_query(prev_req)

## 1 CRM

In [5]:
crm_req = f"""
SELECT
    billing_account_id,
    date_entered,
    date_qualified,
    account_name,
    first_name,
    last_name,
    title,
    website,
    phone_mobile,
    email
FROM (
    SELECT
        id,
        billing_account_id,
        date_entered,
        date_qualified,
        account_name,
        first_name,
        last_name,
        title,
        website,
        phone_mobile
    FROM (
        SELECT
            id,
            toDate(date_entered) as date_entered,
            toDate(date_modified) as date_qualified,
            status,
            first_name,
            last_name,
            title,
            website,
            phone_mobile,
            account_name
        FROM "{leads_table}"
        WHERE lead_source == 'var'
        and status = 'Converted'
    ) as a
    INNER JOIN (
        SELECT
            DISTINCT
            leads_id,
            billing_account_id
        FROM "{billing_leads_keys}" as a
        INNER JOIN (
            SELECT
                id,
                ba_id as billing_account_id
            FROM "{billing_table}"
        ) as b
        ON a.billingaccounts_id == b.id
    ) as b
    ON a.id == b.leads_id
) as a
LEFT JOIN (
    SELECT
        DISTINCT 
        email_address as email,
        id
    FROM "{leads_table}" as a
    INNER JOIN (
        SELECT
            email_address,
            bean_id
        FROM "{email_leads_keys}" as a
        INNER JOIN (
            SELECT
                email_address,
                id
            FROM "{email_table}"
        ) as b
        ON a.email_address_id == b.id
    ) as b
    ON a.id == b.bean_id
) as b
ON a.id == b.id
FORMAT TabSeparatedWithNames
"""

In [6]:
crm_df = lib.execute_query(crm_req)

In [7]:
crm_df.shape

(152, 10)

In [8]:
add_df = prev_df[~prev_df["billing_account_id"].isin(crm_df["billing_account_id"])]

# 1.5 additional from cube

In [9]:
cube_add_df = lib.execute_query("""
SELECT
    billing_account_id,
    null as date_entered,
    date_qualified,
    account_name,
    first_name,
    last_name,
    null as title,
    null as website,
    phone as phone_mobile,
    user_settings_email as email
FROM "//home/cloud_analytics/cubes/acquisition_cube/cube" as cube
INNER JOIN (
    SELECT 
        billing_account_id,
        toDateTime(min(if (replaceRegexpOne(feature_flags, '.*"var\": ([^,|}]*)[,|}].*', '\\1') like '%true%',
        updated_at, null))) as date_qualified

    FROM "//home/cloud/billing/exported-billing-tables/billing_accounts_history_prod"
    GROUP BY billing_account_id
) as dt
ON cube.billing_account_id == dt.billing_account_id
WHERE isNotNull(billing_account_id)
AND is_var == 'var'
AND event == 'ba_created'
AND billing_account_id != ''
AND puid != ''
AND master_account_id == ''
FORMAT TabSeparatedWithNames
""")

In [10]:
cube_add_df = cube_add_df[~cube_add_df["billing_account_id"].isin(crm_df["billing_account_id"])]
cube_add_df = cube_add_df[~cube_add_df["billing_account_id"].isin(prev_df["billing_account_id"])]

In [11]:
crm_df = lib.concatenate_tables([crm_df, add_df[crm_df.columns], cube_add_df[crm_df.columns]])

In [12]:
crm_df = pd.merge(crm_df, 
                  prev_df[["billing_account_id", "description", 'profile']], 
                  how='left')

In [13]:
def make_num_of_days_from_entered_to_qualified(row):
    try:
        days = (parse(row['date_qualified']) - parse(row['date_entered'])).days
        return str(days)
    except Exception:
        return ''

In [14]:
crm_df["num_of_days_from_entered_to_qualified"] =\
crm_df[['date_entered', 'date_qualified']].apply(
    lambda row: make_num_of_days_from_entered_to_qualified(row), axis=1)

In [15]:
def make_days_to_now(x):
    try:
        days = (parse(lib.get_current_date_as_str()) - parse(x)).days
        return str(days)
    except Exception:
        return 'тест'

In [16]:
crm_df["days_to_now"] = crm_df['date_qualified'].apply(lambda x: make_days_to_now(x))

In [17]:
def make_weeks(x):
    try:
        return str(round(int(x) / 7, 2))
    except Exception:
        return 'тест'
    
def make_months(x):
    try:
        return str(round(int(x) / 30, 2))
    except Exception:
        return 'тест'

In [18]:
crm_df["weeks_to_now"] = crm_df["days_to_now"].apply(lambda x: make_weeks(x))
crm_df["months_to_now"] = crm_df["days_to_now"].apply(lambda x: make_months(x))

In [19]:
crm_df.head()

,billing_account_id,date_entered,date_qualified,account_name,first_name,last_name,title,website,phone_mobile,email,description,profile,num_of_days_from_entered_to_qualified,days_to_now,weeks_to_now,months_to_now
0,dn28h2nd3ii9n07ce3tu,2020-07-06,2020-09-02,cloud motion ltd,Екатерина,Листкова,CTO,cloudmotionltd.com,+33 06 52 72 71 79,sales@cloudmotionltd.com,NaN,NaN,58,1,0.14,0.03
1,dn28h2nd3ii9n07ce3tu,2020-07-06,2020-09-02,cloud motion ltd,Екатерина,Листкова,CTO,cloudmotionltd.com,+33 06 52 72 71 79,cloudmotionltd@yandex.ru,NaN,NaN,58,1,0.14,0.03
2,dn2trl71hgk4sr8n1jj8,2020-01-13,2020-01-31,"ООО ""ИТБ""",Александр,Кузнецов,Директор,gtbrain.ru,89043022313,a-s-k74@yandex.ru,"Компания является многопрофильной, занимается ...",???,18,216,30.86,7.2
3,dn222pnhq7jj4jl2pfe1,2020-05-15,2020-08-11,"ООО ""Аджайло""",Иван,Сысоев,Директор,agilo.ru,+79883120820,ivan@agilo.ru,NaN,NaN,88,23,3.29,0.77
4,dn2rm9r5h8udb84gi46r,2020-05-06,2020-05-07,"ООО ""Чек Офис""",Валентин,Колесов,Генеральный диретор,https://checkoffice.ru/,+79153460214,v.m@checkoffice.ru,NaN,NaN,1,119,17.0,3.97


In [20]:
res_table = pd.merge(core_df, crm_df, on='billing_account_id', how='inner')

## 2 Backoffice + Cube + Spark

In [21]:
main_req = """
SELECT
    *
FROM (
    SELECT 
        billing_account_id,
        assumeNotNull(puid) as puid
    FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
    WHERE isNotNull(billing_account_id)
    AND is_var == 'var'
    AND event == 'ba_created'
    AND billing_account_id != ''
    AND puid != ''
    AND master_account_id == ''
) as cube_info
INNER JOIN (
    SELECT
        CAST(assumeNotNull(passport_id) as String) as puid,
        argMax(email, dt) as backoffice_email,
        argMax(phone, dt) as backoffice_phone,
        argMax(name, dt) as backoffice_name
    FROM "//home/cloud_analytics/import/balance/balance_persons"
    WHERE puid != ''
    GROUP BY puid
) as backoiffice_info
using puid
FORMAT TabSeparatedWithNames
"""

In [22]:
df = lib.execute_query(main_req)

In [23]:
df.head()

,billing_account_id,puid,backoffice_email,backoffice_phone,backoffice_name
0,dn2jdakdi2vtdmi8eeav,997560560,man.dxdo@yandex.ru,79211010021,ООО «Ди-Икс-До»
1,dn2aj57416v1p9587u7d,1130000000530703,info@internet-expert.ru,8 (499) 110-88-53,Интернет-Эксперт
2,dn2megvp3lnh8bp093uj,1130000027816971,info@cloudclan.ru,79647097942,КЛАУДКЛАН
3,dn2sa9q2jtepdg1h3unb,256079934,uvereshchagina@kserv.pro,88332777142 доб.238,ТЕЛЕСНАБ
4,dn25qub0s48kcoqcjhs4,1130000031523459,deploy@stroysd.ru,+79039889511,"ООО ""Система"""


In [24]:
df["puid"] = df["puid"].astype(str)

In [25]:
res_table = pd.merge(res_table, df, on='billing_account_id', how='left')

In [26]:
res_table.shape

(258, 28)

## 4 Consumption + accounts

## 4.1

In [27]:
accounts_history_req = f"""
SELECT
    master_account_id,
    sum_one_month_sub_accounts as one_month_sub_accounts,
    sum_two_month_sub_accounts - sum_one_month_sub_accounts as two_month_sub_accounts,
    sum_three_month_sub_accounts - sum_two_month_sub_accounts as three_month_sub_accounts,
    sum_four_month_sub_accounts - sum_three_month_sub_accounts as four_month_sub_accounts,
    sum_five_month_sub_accounts - sum_four_month_sub_accounts as five_month_sub_accounts,
    sum_six_month_sub_accounts - sum_five_month_sub_accounts as six_month_sub_accounts,
    sum_six_plus_month_sub_accounts - sum_six_month_sub_accounts as six_plus_month_sub_accounts,
    sum_six_plus_month_sub_accounts as all_sub_accounts_number
FROM (
    SELECT
        master_account_id,
        length(groupUniqArray(IF(toDate(event_time) < addDays(toDate(date_qualified), 30), b.billing_account_id, null))) as sum_one_month_sub_accounts,
        length(groupUniqArray(IF(toDate(event_time) < addDays(toDate(date_qualified), 60), b.billing_account_id, null))) as sum_two_month_sub_accounts,
        length(groupUniqArray(IF(toDate(event_time) < addDays(toDate(date_qualified), 90), b.billing_account_id, null))) as sum_three_month_sub_accounts,
        length(groupUniqArray(IF(toDate(event_time) < addDays(toDate(date_qualified), 4 * 30), b.billing_account_id, null))) as sum_four_month_sub_accounts,
        length(groupUniqArray(IF(toDate(event_time) < addDays(toDate(date_qualified), 5 * 30), b.billing_account_id, null))) as sum_five_month_sub_accounts,
        length(groupUniqArray(IF(toDate(event_time) < addDays(toDate(date_qualified), 6 * 30), b.billing_account_id, null))) as sum_six_month_sub_accounts,
        length(groupUniqArray(billing_account_id)) as sum_six_plus_month_sub_accounts
    FROM "//home/cloud_analytics/cubes/acquisition_cube/cube" as a
    INNER JOIN (
        SELECT
            billing_account_id,
            date_qualified
        FROM (
            SELECT
                billing_account_id,
                ba_time,
                if (isNotNull(date_qualified), date_qualified, ba_time) as date_qualified
            FROM (
                SELECT 
                    billing_account_id,
                    toDate(event_time) as ba_time
                FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
                WHERE isNotNull(billing_account_id)
                AND is_var == 'var'
                AND event == 'ba_created'
                AND billing_account_id != ''
                AND puid != ''
                AND master_account_id == ''
            ) as a
            LEFT JOIN (
                SELECT
                    billing_account_id,
                    date_qualified
                FROM (
                    SELECT
                        id,
                        toDate(date_modified) as date_qualified
                    FROM "{leads_table}"
                    WHERE lead_source == 'var'
                    and status = 'Converted'
                ) as a
                INNER JOIN (
                    SELECT
                        DISTINCT
                        leads_id,
                        billing_account_id
                    FROM "{billing_leads_keys}" as a
                    INNER JOIN (
                        SELECT
                            id,
                            ba_id as billing_account_id
                        FROM "{billing_table}"
                    ) as b
                    ON a.billingaccounts_id == b.id
                ) as b
                ON a.id == b.leads_id
            ) as b
            ON a.billing_account_id == b.billing_account_id
        )
    ) as b
    ON a.master_account_id == b.billing_account_id
    WHERE billing_account_id != ''
    GROUP BY master_account_id
)
FORMAT TabSeparatedWithNames
"""

In [28]:
accounts_history_df = lib.execute_query(accounts_history_req)

In [29]:
accounts_history_df.head()

,master_account_id,one_month_sub_accounts,two_month_sub_accounts,three_month_sub_accounts,four_month_sub_accounts,five_month_sub_accounts,six_month_sub_accounts,six_plus_month_sub_accounts,all_sub_accounts_number
0,dn2jjoul6v4g892bncoa,1,0,0,0,0,0,4,5
1,dn2jq936m4sh4ehh8h41,1,0,0,0,0,0,7,8
2,dn22ijjfmehsfifkudjc,1,0,0,0,0,0,0,1
3,dn2r3535u9jbjdgv693t,1,0,0,0,0,0,0,1
4,dn2s9c8ppd01l2458t2h,1,0,0,0,0,0,2,3


In [30]:
accounts_history_df.columns = ['billing_account_id', 'one_month_sub_accounts',
       'two_month_sub_accounts', 'three_month_sub_accounts',
       'four_month_sub_accounts', 'five_month_sub_accounts',
       'six_month_sub_accounts', 'six_plus_month_sub_accounts', "all_sub_accounts_number"]

In [31]:
res_table = pd.merge(res_table, accounts_history_df, on='billing_account_id', how='left')

In [32]:
for column in accounts_history_df.columns:
    res_table[column].fillna(0, inplace=True)

In [33]:
res_table.tail()

,billing_account_id,billing_created_time,mail_billing,mail_tech,mail_testing,mail_feature,mail_promo,mail_info,mail_event,date_entered,...,backoffice_phone,backoffice_name,one_month_sub_accounts,two_month_sub_accounts,three_month_sub_accounts,four_month_sub_accounts,five_month_sub_accounts,six_month_sub_accounts,six_plus_month_sub_accounts,all_sub_accounts_number
253,dn2q97qq6iant0n4l061,2019-03-21,1,1,1,1,1,1,1,2019-03-21 00:00:00,...,79371497712,"ООО “КОМПЛЕКС ЛАБС""",1.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0
254,dn286fi6ecjok3amps8u,2019-06-21,1,1,1,1,1,1,1,2019-04-29 17:30:21.153000,...,"79045800258,+7 (999) 333-24-73",Партнерский,1.0,0.0,0.0,0.0,0.0,0.0,5.0,6.0
255,dn2mrimr30h9q41j2grr,2020-04-23,1,1,1,1,1,1,1,2020-05-06,...,79996490303,ИП Возненко А.Б.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
256,dn2vgcc42rqjrpcmsd0p,2019-08-05,1,1,1,1,1,1,1,2019-10-01 00:00:00,...,+7 (495) 258-32-34,Управляющая компания ГлоуБайт,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
257,dn2hnoq7thg31ng16uvn,2019-09-16,1,1,0,0,0,0,0,2019-09-24 00:00:00,...,79281300006,ВебДизайн-Юг,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
res_table.shape

(258, 36)

## 4.2

In [35]:
names_get_req = """
SELECT CAST(toYear(toStartOfQuarter(NOW())) as String) || '-Q' ||
    CAST((toMonth(toStartOfQuarter(NOW())) - 1) / 3  + 1 as String)
"""

In [36]:
intervals = list(reversed(["addQuarters(toStartOfQuarter(NOW()), -4)",
             "addQuarters(toStartOfQuarter(NOW()), -3)",
             "addQuarters(toStartOfQuarter(NOW()), -2)",
             "addQuarters(toStartOfQuarter(NOW()), -1)",
             "toStartOfQuarter(NOW())",
             "addQuarters(toStartOfQuarter(NOW()), 1)"]))

In [37]:
def iterate_by_two(intervals):
    for i in range(len(intervals) - 1):
        yield intervals[i + 1], intervals[i]

In [38]:
names_get_req = "SELECT "
for val in intervals:
    names_get_req += f"""
    CAST(toYear({val}) as String) || '_Q' ||
    CAST((toMonth({val}) - 1) / 3  + 1 as String),
    """
names_get_req = names_get_req[:-6]

In [39]:
col_names = lib.execute_query(names_get_req)

In [40]:
col_names = col_names.columns.tolist()[1:]

In [41]:
col_names

['2020_Q3', '2020_Q2', '2020_Q1', '2019_Q4', '2019_Q3']

In [42]:
part_req = ""
for col, (left, right) in zip(col_names, iterate_by_two(intervals)):
    part_req += f"""    SUM(if(toDate(event_time) >= {left} 
    and toDate(event_time) < {right}, 
    real_consumption_vat, 0)) as billing_consumption_{col},\n\n"""

In [43]:
req = f"""
SELECT
    {part_req}
    billing_account_id
FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
GROUP BY billing_account_id
HAVING billing_account_id != ''
FORMAT TabSeparatedWithNames
"""

In [44]:
consumption_var_df = lib.execute_query(req)

In [45]:
consumption_var_df.head()

,billing_consumption_2020_Q3,billing_consumption_2020_Q2,billing_consumption_2020_Q1,billing_consumption_2019_Q4,billing_consumption_2019_Q3,billing_account_id
0,0.0,0.0,0.0,0.0,31.166805,dn2vchetqgl7ssbgmc3u
1,0.0,0.0,0.0,0.0,0.000000,dn22of3d9v256epmlaim
2,0.0,0.0,0.0,0.0,0.000000,dn28ngtukfpr4s564cbt
3,0.0,0.0,0.0,0.0,0.000000,dn2h6c2inq3duk6c9dd7
4,0.0,0.0,0.0,0.0,0.000000,dn223717iei032gug0pp


In [46]:
consumption_var_df.shape

(118893, 6)

In [47]:
res_table = pd.merge(res_table, consumption_var_df, on='billing_account_id', how='left')

In [48]:
res_table.head()

,billing_account_id,billing_created_time,mail_billing,mail_tech,mail_testing,mail_feature,mail_promo,mail_info,mail_event,date_entered,...,four_month_sub_accounts,five_month_sub_accounts,six_month_sub_accounts,six_plus_month_sub_accounts,all_sub_accounts_number,billing_consumption_2020_Q3,billing_consumption_2020_Q2,billing_consumption_2020_Q1,billing_consumption_2019_Q4,billing_consumption_2019_Q3
0,dn2jdakdi2vtdmi8eeav,2019-12-13,1,1,1,1,1,1,1,2019-12-12,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,dn2mhinom9sqcgbuhs19,2020-03-11,1,1,1,1,1,1,1,2020-04-29,...,0.0,0.0,0.0,0.0,0.0,0.000000,17.419110,0.000000,0.000000,0.000000
2,dn2mhinom9sqcgbuhs19,2020-03-11,1,1,1,1,1,1,1,2020-03-02,...,0.0,0.0,0.0,0.0,0.0,0.000000,17.419110,0.000000,0.000000,0.000000
3,dn25qub0s48kcoqcjhs4,2018-10-21,1,1,1,1,1,1,1,2019-11-06 17:09:19.274000,...,0.0,0.0,0.0,1.0,1.0,14019.635889,25243.598562,24377.686411,12595.301344,8855.373127
4,dn292g24vvfq8bko83nb,2020-06-15,1,1,1,1,1,1,1,NaN,...,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [49]:
part_req = ""
for col, (left, right) in zip(col_names, iterate_by_two(intervals)):
    part_req += f"""    SUM(if(toDate(event_time) >= {left} 
    and toDate(event_time) < {right}, 
    real_consumption_vat, 0)) as cons_subaccount_{col},
    length(groupUniqArray(if(toDate(event_time) >= {left} 
    and toDate(event_time) < {right}, 
    billing_account_id, null))) as subaccount_num_{col},
    cons_subaccount_{col} / (subaccount_num_{col} + 1e-5)
    as avg_cons_by_one_subaccount_{col},
    \n\n"""

In [50]:
req = f"""
SELECT
    {part_req}
    master_account_id
FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
GROUP BY master_account_id
HAVING master_account_id != ''
FORMAT TabSeparatedWithNames
"""

In [51]:
subaccounts_consumption_var_df = lib.execute_query(req)

In [52]:
subaccounts_consumption_var_df.head()

,cons_subaccount_2020_Q3,subaccount_num_2020_Q3,avg_cons_by_one_subaccount_2020_Q3,cons_subaccount_2020_Q2,subaccount_num_2020_Q2,avg_cons_by_one_subaccount_2020_Q2,cons_subaccount_2020_Q1,subaccount_num_2020_Q1,avg_cons_by_one_subaccount_2020_Q1,cons_subaccount_2019_Q4,subaccount_num_2019_Q4,avg_cons_by_one_subaccount_2019_Q4,cons_subaccount_2019_Q3,subaccount_num_2019_Q3,avg_cons_by_one_subaccount_2019_Q3,master_account_id
0,55303.623253,4,13825.871248,40006.289543,4,10001.547382,36843.724105,4,9210.907999,13488.060499,3,4496.005180,0.000000,1,0.000000,dn2jjoul6v4g892bncoa
1,220407.403475,6,36734.506022,405956.044532,8,50744.442136,325212.838338,4,81303.006327,393255.693731,4,98313.677649,89682.859735,4,22420.658882,dn2jq936m4sh4ehh8h41
2,8642.909788,1,8642.823359,12910.926889,1,12910.797781,11038.594423,1,11038.484038,9354.227865,1,9354.134324,7884.362942,1,7884.284099,dn22ijjfmehsfifkudjc
3,869.712320,1,869.703623,1319.382183,1,1319.368989,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,dn2r3535u9jbjdgv693t
4,4142.049386,2,2071.014338,7260.819533,3,2420.265110,9446.918634,3,3148.962382,0.000000,1,0.000000,0.000000,0,0.000000,dn2s9c8ppd01l2458t2h


In [53]:
cols = subaccounts_consumption_var_df.columns.tolist()
cols = cols[:-1] + ["billing_account_id"]
subaccounts_consumption_var_df.columns = cols

In [54]:
res_table = pd.merge(res_table, 
                     subaccounts_consumption_var_df,
                     on='billing_account_id', how='left')

In [55]:
for column in cols[:-1]:
    res_table[column].fillna(0, inplace=True)

In [56]:
res_table = res_table.sort_values(by='billing_account_id')

In [57]:
res_table['billing_account_id'].unique().shape

(248,)

In [58]:
res_table.shape

(258, 56)

In [59]:
rows = []
last_billing = ''
for _, row in res_table.iterrows():
    if row['billing_account_id'] == last_billing:
        continue
    else:
        last_billing = row['billing_account_id']
        rows.append(row.to_dict())
        
new_df = pd.DataFrame(rows)

In [60]:
new_df.shape

(248, 56)

In [61]:
lib.save_table("var_info", "//home/cloud_analytics/lunin-dv/dashboard_tables", 
               new_df)

In [62]:
lib.save_table_from_yt_to_grafana("//home/cloud_analytics/lunin-dv/dashboard_tables/var_info",
                              "cloud_analytics.var_info",
                              sort_col = 'billing_account_id')

2020-09-03 20:57:56,187	INFO	Transfer task started: https://transfer-manager.yt.yandex-team.ru/task?id=39bb3f48-a570ab4e-1189b10c-ac5f6bd6&tab=details&backend=production
